In [1]:
import pandas as pd
import sys
import numpy as np
import json

In [2]:
archival_info_string = open("archival_info_locations.json", "r").read()
archival_info = json.loads(archival_info_string)

In [3]:
def get_locations(invno):
    
    try:
        locations = archival_info[str(invno)]['locations']
        output = ''
        for x in locations:
            output = output + ';' + x
        
        return output.strip(';')
    
    except KeyError:
        return " "

In [4]:
def build_mapping(df):
    output = {}
    for i, row in df.iterrows():
        output[df.loc[i, "File Name"]] = df.loc[i, "inv_no"]
        
    return output

In [5]:
def clean_nan_rows(df):
    condition = df.iloc[:, 1].apply(identify_document_name_rows) & df.iloc[:, 2:].isna().all(axis=1)
    output = df[~condition]
    return output

In [6]:
def identify_document_name_rows(cell):
    #easiest marker right now is the .png filename
    if type(cell) == str:
        if type(cell) == float:
            return False
        elif '.png' in cell:
            return True
        elif '.jpg' in cell:
            return True
        else:
            return False

In [7]:
def safe_split_page_nos(row):
    """ Wrapper to handle potential unpacking issues """
    try:
        start, end = split_page_nos(row)
        return start, end
    except Exception as e:
        print(f"Error processing row: {row}, Error: {e}")
        return None, None  # Return a safe default

In [8]:
def rename_col_by_index(dataframe, index_mapping):
    dataframe.columns = [index_mapping.get(i, col) for i, col in enumerate(dataframe.columns)]
    return dataframe

In [9]:
def add_invno(value, png_mapping):
    return png_mapping[value]

In [10]:
def main():
    #read the excel file
    df = pd.read_excel("TANAP merged/gen geo coverage.xlsx")
    
    #remove any rows that don't contain information other than the scan title
       
    #add locations
    df['GEOGRAPHICAL COVERAGE OF INV. NUMBER'] = df['INVENTORY NUMBER'].apply(get_locations)
    
    df.to_excel('tanap geo coverage.xlsx')
    
    return df

In [11]:
mapping_df = build_mapping(pd.read_csv('png_mapping.csv'))

In [12]:
#add_invno('224313.1_NL-HaNA_2.14.03_2461-14_contrast.png', mapping_df)

In [13]:
def split_page_nos(cell):
    dividers = [' a ', '-', '—']
    
    if isinstance(cell, str):
        found = False
        for divider in dividers:
            if divider in cell:
                found = True
                parts = [part.strip() for part in cell.split(divider)]
                # Ensure there are exactly two parts after splitting
                if len(parts) == 2:
                    start_page, end_page = parts
                else:
                    # Fallback if there's unexpected behavior
                    start_page = end_page = str(cell)
                break
        else:
            # No divider was found, use the cell itself
            start_page = end_page = str(cell)
    else:
        start_page = end_page = str(cell)
    
    return start_page, end_page

In [14]:
main()

,ID (TANAP),ID (GLOBALISE),DESCRIPTION,YEAR (EARLIEST),YEAR (LATEST),YEARS (ALL),SETTLEMENT,LOCATION,GEOGRAPHICAL COVERAGE OF INV. NUMBER,INVENTORY NUMBER,SECTION,FOLIONUMBER (START OF DOCUMENT),FOLIONUMBER (END OF DOCUMENT),FOLIONUMBERS (AS THEY APPEAR IN TYPOSCRIPT),DOCUMENT TYPE,LINK-2,LINK
0,1,NaN,Copie verklaringh verleent door de Addeponaers...,1694,1694,1694,Ceylon,Tutucorin|Jaffanapatnam,Ceylon,1543,NaN,697,698,NaN,"RUBRIEK:Brieven, rapporten en instructies verz...",NL-HaNA/1.04.02/1543///Katern: Ceylon Pagina ...,244ea5cc-9aed-102e-9032-0050569c51dd
1,2,NaN,Lijst der te Batavia in 1784 geweest zijnde Ch...,1784,1784,1784,Batavia,NaN,Batavia,3656,NaN,978,979,NaN,RUBRIEK:Stukken betreffende schepen en equipag...,NL-HaNA/1.04.02/3656///Katern: Batavia Pagina...,47deaf06-b00e-102e-9c8f-0050569c51dd
2,3,NaN,Resoluties genomen in rade van politie tot Col...,1733,1733,1733,Ceylon,NaN,,8959,NaN,438,471,NaN,"RUBRIEK:Brieven, rapporten en instructies verz...",NL-HaNA/1.04.02/8959///Katern: Ceylon Pagina ...,47f794e4-b00e-102e-9c8f-0050569c51dd
3,4,NaN,Copie missive van Jan Dircksz. Lam uijt de bai...,1617,1617,1617,"Azië, plaatsen in -",NaN,,1064,NaN,143,144,NaN,"RUBRIEK:Overige brieven, rapporten en instruct...","NL-HaNA/1.04.02/1064///Katern: Azië, plaatsen ...",481869c6-b00e-102e-9c8f-0050569c51dd
4,5,NaN,Extract uijt de consideratien vanden raad ordi...,1738,1739,1738|1739,Coromandel,NaN,Bengalen;Coromandel,2489,NaN,479,485,NaN,"RUBRIEK:Overige brieven, rapporten en instruct...",NL-HaNA/1.04.02/2489///Katern: Coromandel Pag...,482c310e-b00e-102e-9c8f-0050569c51dd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170127,170128,NaN,Duplicaat register der verzondene papieren met...,1786,1786,1786,Batavia,NaN,,7590,NaN,789,791,NaN,"RUBRIEK:Brieven, rapporten en instructies ontv...",NL-HaNA/1.04.02/7590///Katern: Batavia Pagina...,89205aca-ab66-102e-8314-0050569c51dd
170128,170129,NaN,Copia registers met de schepen Texelstroom en ...,1786,1786,1786,Batavia,NaN,,7590,NaN,793,798,NaN,"RUBRIEK:Brieven, rapporten en instructies ontv...",NL-HaNA/1.04.02/7590///Katern: Batavia Pagina...,89206236-ab66-102e-8314-0050569c51dd
170129,170130,NaN,Lijst van de officieren en passagiers met het ...,1787,1787,1787,Batavia,NaN,,7590,NaN,799,801,NaN,RUBRIEK:Stukken betreffende personeel en bevol...,NL-HaNA/1.04.02/7590///Katern: Batavia Pagina...,892069c0-ab66-102e-8314-0050569c51dd
170130,170131,NaN,Origineele missive van hunne hoogedelheeden aa...,1787,1787,1787,Batavia,NaN,,7590,NaN,803,807,NaN,"RUBRIEK:Brieven, rapporten en instructies ontv...",NL-HaNA/1.04.02/7590///Katern: Batavia Pagina...,89207154-ab66-102e-8314-0050569c51dd
